In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 6.3 MB/s eta 0:00:00


In [3]:
!pip install gensim==4.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 46.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-4.1.2-cp310-cp310-linux_x86_64.whl size=25996775 sha256=9cda924ab93b1b3996d3dd790874daffd96a6f3cf35a23c6725be8dc353b7d7b
  Stored in directory: /root/.cache/pip/wheels/13/35/4e/dca2954de21981d0a137ff930239f0767403a617e32f19f04f
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


In [4]:
!pip install py_vncorenlp

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.5 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4306 sha256=7de7a431ed7e7d57d2e010d0661d3265984c46c9a7c0d10fdaa32609f47d206d
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [5]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow_addons.text import crf_log_likelihood, crf_decode
from tensorflow.keras.preprocessing.text import Tokenizer, tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, TimeDistributed, Input, BatchNormalization, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import initializers, Sequential
import tensorflow.keras.optimizers as Optimizer

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
<ipython-input-5-b47b39284be7>:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [6]:
from sklearn.model_selection import train_test_split
import pickle
import string

In [7]:
import py_vncorenlp

py_vncorenlp.download_model(save_dir='/content/')
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content/')

In [8]:
cd /content/drive/MyDrive/Colab Notebooks/NLP/CRF

/content/drive/MyDrive/Colab Notebooks/NLP/CRF


#DATA

In [9]:
org_data = pd.read_csv("Data/VDT/id_ner.csv")
with open("Data/VDT/i2l.pkl", "rb") as file_:
  i2l = pickle.load(file_)
for key, value in i2l.items():
  i2l[key] = "_".join(value.split(' '))
org_data = org_data.drop(columns = ['Unnamed: 0'])
org_data['tokens'] = org_data['tokens'].apply(lambda x: ' '.join(eval(x)))
org_data['ner_tags'] = org_data['ner_tags'].apply(lambda x: eval(x))

In [10]:
i2l

{0: 'I_<tên_cụm_chỉ_tiêu>',
 1: 'I_<đơn_vị>',
 2: 'B_<tên_tổng_công_ty>',
 3: 'I_<tên_chỉ_tiêu>',
 4: 'B_<quý>',
 5: 'B_<tên_chỉ_tiêu>',
 6: 'O',
 7: 'B_<năm>',
 8: 'B_<đơn_vị>',
 9: 'B_<năm_m>',
 10: 'B_<tháng>',
 11: 'B_<tháng_1>',
 12: 'B_<tên_cụm_chỉ_tiêu>'}

In [11]:
def handle_punc(text):
    if text[-1] in string.punctuation and text[-2] != ' ':
      text = text[:-1]
    return text

org_data['tokens'] = org_data['tokens'].apply(lambda x: handle_punc(x))
org_data['ner_tags'] = org_data['ner_tags'].apply(lambda x: [i2l[each] for each in x])

In [12]:
train_words = [each.split(' ') for each in org_data['tokens'].values]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(org_data['tokens'], org_data['ner_tags'], test_size=0.05, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05, random_state=1)

In [14]:
train_data, val_data, test_data = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
train_data['tokens'], train_data['ner_tags'] = X_train, y_train
val_data['tokens'], val_data['ner_tags'] = X_val, y_val
test_data['tokens'], test_data['ner_tags'] = X_test, y_test

In [15]:
train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)

#CRF

In [16]:
class CRF(L.Layer):
    def __init__(self,
                 output_dim,
                 sparse_target=True,
                 **kwargs):
        """
        Args:
            output_dim (int): the number of labels to tag each temporal input.
            sparse_target (bool): whether the the ground-truth label represented in one-hot.
        Input shape:
            (batch_size, sentence length, output_dim)
        Output shape:
            (batch_size, sentence length, output_dim)
        """
        super(CRF, self).__init__(**kwargs)
        self.output_dim = int(output_dim)
        self.sparse_target = sparse_target
        self.input_spec = L.InputSpec(min_ndim=3)
        self.supports_masking = False
        self.sequence_lengths = None
        self.transitions = None

    def build(self, input_shape):
        assert len(input_shape) == 3
        f_shape = tf.TensorShape(input_shape)
        input_spec = L.InputSpec(min_ndim=3, axes={-1: f_shape[-1]})

        if f_shape[-1] is None:
            raise ValueError('The last dimension of the inputs to `CRF` '
                             'should be defined. Found `None`.')
        if f_shape[-1] != self.output_dim:
            raise ValueError('The last dimension of the input shape must be equal to output'
                             ' shape. Use a linear layer if needed.')
        self.input_spec = input_spec
        self.transitions = self.add_weight(name='transitions',
                                           shape=[self.output_dim, self.output_dim],
                                           initializer='glorot_uniform',
                                           trainable=True)
        self.built = True

    def compute_mask(self, inputs, mask=None):
        # Just pass the received mask from previous layer, to the next layer or
        # manipulate it if this layer changes the shape of the input
        return mask

    def call(self, inputs, sequence_lengths=None, training=None, **kwargs):
        sequences = tf.convert_to_tensor(inputs, dtype=self.dtype)
        if sequence_lengths is not None:
            assert len(sequence_lengths.shape) == 2
            assert tf.convert_to_tensor(sequence_lengths).dtype == 'int32'
            seq_len_shape = tf.convert_to_tensor(sequence_lengths).get_shape().as_list()
            assert seq_len_shape[1] == 1
            self.sequence_lengths = K.flatten(sequence_lengths)
        else:
            self.sequence_lengths = tf.ones(tf.shape(inputs)[0], dtype=tf.int32) * (
                tf.shape(inputs)[1]
            )

        viterbi_sequence, _ = crf_decode(sequences,
                                         self.transitions,
                                         self.sequence_lengths)
        output = K.one_hot(viterbi_sequence, self.output_dim)
        return K.in_train_phase(sequences, output)

    @property
    def loss(self):
        def crf_loss(y_true, y_pred):
            y_pred = tf.convert_to_tensor(y_pred, dtype=self.dtype)
            log_likelihood, self.transitions = crf_log_likelihood(
                y_pred,
                tf.cast(K.argmax(y_true), dtype=tf.int32) if self.sparse_target else y_true,
                self.sequence_lengths,
                transition_params=self.transitions,
            )
            return tf.reduce_mean(-log_likelihood)
        return crf_loss

    @property
    def accuracy(self):
        def viterbi_accuracy(y_true, y_pred):
            # -1e10 to avoid zero at sum(mask)
            mask = K.cast(
                K.all(K.greater(y_pred, -1e10), axis=2), K.floatx())
            shape = tf.shape(y_pred)
            sequence_lengths = tf.ones(shape[0], dtype=tf.int32) * (shape[1])
            y_pred, _ = crf_decode(y_pred, self.transitions, sequence_lengths)
            if self.sparse_target:
                y_true = K.argmax(y_true, 2)
            y_pred = K.cast(y_pred, 'int32')
            y_true = K.cast(y_true, 'int32')
            corrects = K.cast(K.equal(y_true, y_pred), K.floatx())
            return K.sum(corrects * mask) / K.sum(mask)
        return viterbi_accuracy

    def compute_output_shape(self, input_shape):
        tf.TensorShape(input_shape).assert_has_rank(3)
        return input_shape[:2] + (self.output_dim,)

    def get_config(self):
        config = {
            'output_dim': self.output_dim,
            'sparse_target': self.sparse_target,
            'supports_masking': self.supports_masking,
            'transitions': K.eval(self.transitions)
        }
        base_config = super(CRF, self).get_config()
        return dict(base_config, **config)

#MODEL FASTTEXT - Tokenizer

In [17]:
from gensim.models import FastText
trained = True

In [18]:
from gensim.models import FastText
# model_fasttext = FastText(vector_size=100, window=5,
#                           workers=4, sg=1, negative=5, alpha=0.005)
# model_fasttext.build_vocab(train_words)
# model_fasttext.train(train_words, total_examples=model_fasttext.corpus_count,
#                      epochs=90)

In [19]:
def train_model_fasttext(train_data, size=100, window=5, min_count=3, negative=5, sg=1, alpha=0.005, epoch=90):
    # train_data = [each.split() for each in train_data]
    model_fasttext = FastText(vector_size=size, window=window, min_count=min_count,
                              workers=4, sg=1, negative=negative, alpha=alpha)
    model_fasttext.build_vocab(train_data)
    model_fasttext.train(train_data, total_examples=model_fasttext.corpus_count, epochs=epoch)
    return model_fasttext

if trained:
    model_fasttext = FastText.load('Data/Fasttext/model_fasttext_gensim.bin')
else:
    model_fasttext = train_model_fasttext(train_words)
    model_fasttext.save('Data/Fasttext/model_fasttext_gensim.bin')

In [20]:
tokenizer = Tokenizer(num_words=len(model_fasttext.wv.key_to_index), lower=True, filters="", oov_token='-OOV-')
tokenizer.fit_on_texts([*train_data['tokens'], *val_data['tokens']])
word_index = tokenizer.word_index
emb_mean, emb_std = -0.5,0.5
embed_size = 100 #Kích thước vector biểu diễn 1 từ
nb_words = len(word_index) + 1
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    if word in model_fasttext.wv.key_to_index:
        embedding_matrix[i] = model_fasttext.wv.get_vector(word)

In [21]:
import pickle

# saving
with open('/content/drive/MyDrive/Colab Notebooks/NLP/CRF/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('/content/drive/MyDrive/Colab Notebooks/NLP/CRF/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [22]:
max_length = 100

In [23]:
X_train = tokenizer.texts_to_sequences(train_data['tokens'])
X_val = tokenizer.texts_to_sequences(val_data['tokens'])
X_test = tokenizer.texts_to_sequences(test_data['tokens'])

X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')

In [24]:
tag_tokenizer = Tokenizer(filters="", lower=False)
tag_tokenizer.fit_on_texts(train_data['ner_tags'].apply(lambda x: ' '.join(x)))
tag_index = tag_tokenizer.word_index
tag_size = len(tag_index) + 1

y_train = tag_tokenizer.texts_to_sequences(train_data['ner_tags'].apply(lambda x: ' '.join(x)))
y_val = tag_tokenizer.texts_to_sequences(val_data['ner_tags'].apply(lambda x: ' '.join(x)))
y_test = tag_tokenizer.texts_to_sequences(test_data['ner_tags'].apply(lambda x: ' '.join(x)))

y_train = pad_sequences(y_train, maxlen=max_length, padding='post')
y_val = pad_sequences(y_val, maxlen=max_length, padding='post')
y_test = pad_sequences(y_test, maxlen=max_length, padding='post')

In [25]:
y_train = [to_categorical(i, num_classes=tag_size) for i in y_train]
y_val = [to_categorical(i, num_classes=tag_size) for i in y_val]
y_test = [to_categorical(i, num_classes=tag_size) for i in y_test]
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)
y_test = np.asarray(y_test)

In [26]:
import pickle

# saving
with open('/content/drive/MyDrive/Colab Notebooks/NLP/CRF/tag_tokenizer.pickle', 'wb') as handle:
    pickle.dump(tag_tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('/content/drive/MyDrive/Colab Notebooks/NLP/CRF/tag_tokenizer.pickle', 'rb') as handle:
    tag_tokenizer = pickle.load(handle)

#MODEL

In [27]:
def create_model(embeddings_matrix, vocab_size, embedding_dim, max_length):
    crf = CRF(len(tag_index), sparse_target=True)
    input = Input(shape = (max_length, ), dtype='int32', name='input_text')
    x = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                  weights=[embedding_matrix])(input)
    x = Bidirectional(LSTM(units=max_length, return_sequences=True,
                                recurrent_dropout=0.01))(x)
    x = TimeDistributed(Dense(128, activation='relu', kernel_initializer='he_normal'))(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.6)(x)
    x = Dense(len(tag_index), activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.1)(x)
    output = crf(x)
    model_final = Model(input, output)
    model_final.compile(optimizer=Optimizer.Adam(lr=0.001), loss=crf.loss,
                        metrics=[crf.accuracy])

    return model_final

In [28]:
model = create_model(embedding_matrix, nb_words, embed_size, max_length)
model_check = create_model(embedding_matrix, nb_words, embed_size, max_length)

In [29]:
#model.fit(X_train, y_train, batch_size=512, epochs=5, validation_data = (X_val, y_val), verbose=1)

In [30]:
  # model.evaluate(X_test, y_test)

In [31]:
# model.save_weights("Data/Best_model/best_weight.hdf5")
# model.save_weights("Data/Best_model/best_weight_1.hdf5") #thêm 1 lớp linear 64, dropout=0.3
# model.save_weights("Data/Fasttext/best_weight.hdf5")

In [32]:
# model.load_weights("Data/Fasttext/best_weight.hdf5")
model.load_weights("Data/Fasttext/best_weight.hdf5")

In [33]:
def get_tags(sequences, tag_index):
    sequence_tags = []
    for sequence in sequences:
        sequence_tag = []
        for categorical in sequence:
            sequence_tag.append(tag_index.get(np.argmax(categorical)))
        sequence_tags.append(sequence_tag)
    return sequence_tags

def predict(model, tag_tokenizer, sent):
    tag_index = tag_tokenizer.word_index
    tag_size = len(tag_index) + 1
    pred = model.predict(sent)
    sequence_tags = get_tags(pred, {i: t for t, i in tag_index.items()})
    for idx, each in enumerate(sequence_tags):
        try:
           idx_cut = each.index(None)
        except:
           idx_cut = len(each) + 1
        sequence_tags[idx] = each[:idx_cut]
    return sequence_tags

In [34]:
res = predict(model, tag_tokenizer, X_test)
# res[20].append('O')

188/188 [==============================] - 34s 147ms/step


In [35]:
df_predict = pd.DataFrame(
    data = {
        "sent": test_data['tokens'],
        "pre": res,
        "true": test_data['ner_tags']
    }
)

In [36]:
arr_true = []
for each in df_predict["true"]:
    arr_true = [*arr_true, *each]

arr_predict = []
for each in res:
    arr_predict = [*arr_predict, *each]

In [37]:
len(arr_true), len(arr_predict)

(151475, 151475)

In [38]:
from sklearn.metrics import classification_report, f1_score
from sklearn.preprocessing import LabelEncoder

In [39]:
le = LabelEncoder().fit(arr_true)
list_label = list(le.classes_)
# list_label.remove('O')

In [40]:
print(classification_report(arr_true, arr_predict, labels=list_label))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                      precision    recall  f1-score   support

             B_<năm>       1.00      0.99      1.00      7735
           B_<năm_m>       0.98      0.97      0.97       284
             B_<quý>       1.00      0.97      0.99      1012
           B_<tháng>       1.00      0.97      0.99      5929
         B_<tháng_1>       0.00      0.00      0.00        65
    B_<tên_chỉ_tiêu>       1.00      0.99      0.99      4123
B_<tên_cụm_chỉ_tiêu>       1.00      1.00      1.00      1611
B_<tên_tổng_công_ty>       1.00      0.97      0.99      2048
          B_<đơn_vị>       1.00      0.61      0.76       159
    I_<tên_chỉ_tiêu>       1.00      1.00      1.00     23933
I_<tên_cụm_chỉ_tiêu>       1.00      1.00      1.00      8279
          I_<đơn_vị>       0.00      0.00      0.00        14
                   O       1.00      1.00      1.00     96283

            accuracy                           1.00    151475
           macro avg       0.84      0.81      0.82    151475
      

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [41]:
f1_score(arr_true, arr_predict, average='macro')

0.8213200717028154

In [ ]:
test_text = "Có thể cho tôi biết đến lần này giá trị thấp nhất cho Tổng đài 18009000 đạt được là bao nhiêu trong trong khoảng thời gian từ n 2021 cho đến th 11 2021"
rdrsegmenter.word_segment(test_text)

['Có_thể cho tôi biết đến lần này giá_trị thấp nhất cho Tổng_đài 18009000 đạt được là bao_nhiêu trong trong khoảng thời_gian từ n 2021 cho đến th 11 2021']

In [ ]:
def match_pair_ner(text, ner):
    dict_ = {}
    text_arr = text.split(' ')
    save_ner, save_words = None, ''
    for i in range(len(text_arr)):
      if ner[i] == 'O' or 'B_' in ner[i] or ('I_' in ner[i] and ner[i - 1] == 'O'):
        if save_ner is not None and save_ner != 'O':
          if save_ner not in dict_:
            dict_[save_ner] = []
          dict_[save_ner].append(save_words.replace('_', ' ').strip())
        save_words = text_arr[i] + ' '
      else:
        save_words += text_arr[i] + ' '
      save_ner = ner[i] if ner[i] == 'O' else ner[i][2:]
    if save_ner is not None and save_ner != 'O':
        if save_ner not in dict_:
            dict_[save_ner] = []
        dict_[save_ner].append(save_words.replace('_', ' ').strip())
    return dict_

def infer_ner(text, model, tokenizer, tag_tokenizer, max_length=100):
    word_segment = rdrsegmenter.word_segment(text)
    res_text = tokenizer.texts_to_sequences(word_segment)
    res_text = pad_sequences(res_text, maxlen=max_length, padding='post')
    res_text = predict(model, tag_tokenizer, res_text)
    print(word_segment)
    dict_ = match_pair_ner(word_segment[0], res_text[0])
    return dict_

In [ ]:
infer_ner(test_text, model, tokenizer, tag_tokenizer)

1/1 [==============================] - 0s 119ms/step
['Có_thể cho tôi biết đến lần này giá_trị thấp nhất cho Tổng_đài 18009000 đạt được là bao_nhiêu trong trong khoảng thời_gian từ n 2021 cho đến th 11 2021']


{'<tên_chỉ_tiêu>': ['Tổng đài 18009000'],
 '<năm>': ['2021', '2021'],
 '<tháng>': ['11']}

In [42]:
df_predict[df_predict['sent'].str.contains("quý")]

,sent,pre,true
5,Tôi muốn biết đến thời_điểm này giá_trị nhỏ nhất chỉ_tiêu Nhóm dịch_vụ sme đạt được là bao_nhiêu cũng như kết_quả dự_báo kpi là trong trong quý 2 năm 2023 đến hết th 6 năm 2023,"[O, O, O, O, O, O, O, O, O, O, B_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, O, B_<năm>, O, O, O, B_<tháng>, O, B_<năm>]","[O, O, O, O, O, O, O, O, O, O, B_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, O, B_<năm>, O, O, O, B_<tháng>, O, B_<năm>]"
6,Tôi có một vấn_đề cần được giải_đáp tập_hợp chỉ_tiêu Tỷ_lệ kết_nối thành_công đến tổng_đài có tổng_thể về các chỉ_tiêu con ra làm_sao từ đầu quý 4 2020 tính đến tháng 10 2020,"[O, O, O, O, O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, B_<tháng>, B_<năm>]","[O, O, O, O, O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, B_<tháng>, B_<năm>]"
7,Trong trong giai_đoạn quý 1 năm 2022 cho đến thg 3 năm 2022 trung_bình với Tổng_đài 18009000,"[O, O, O, O, B_<quý>, O, B_<năm>, O, O, O, B_<tháng>, O, B_<năm>, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>]","[O, O, O, O, B_<quý>, O, B_<năm>, O, O, O, B_<tháng>, O, B_<năm>, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>]"
8,Cả tháng và quý đều nhận được đánh_giá không ổn của chỉ_tiêu Tỷ_lệ hài lòng của KH dịch vụ chuyển phát hay không trong th 3 2022,"[O, O, O, O, O, O, O, O, O, O, O, O, B_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, O, O, O, O, B_<tháng>, B_<năm>]","[O, O, O, O, O, O, O, O, O, O, O, O, B_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, I_<tên_chỉ_tiêu>, O, O, O, O, B_<tháng>, B_<năm>]"
22,Cho tôi biết cụm chỉ_tiêu Tỷ_lệ xử lý thành công tại bàn dịch vụ chuyển phát có giá_trị tối_thiểu ra sao và đề_cập tên chỉ_tiêu nhận được giá_trị này trong trong khoảng thời_gian từ quý 1 2020 tính đến thg 1 2020,"[O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, B_<tháng>, B_<năm>]","[O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, B_<tháng>, B_<năm>]"
...,...,...,...
5972,Trong khoảng thời_gian từ quý 3 2021 bắt_đầu đến hết t 8 2021 tui muốn biết chi_tiết hơn của cụm chỉ_tiêu Tỷ_lệ kết_nối thành_công đến tổng_đài với những chỉ_tiêu đã được đánh_giá là hoàn_thành liên_quan VDS,"[O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, O, B_<tháng>, B_<năm>, O, O, O, O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, B_<tên_tổng_công_ty>]","[O, O, O, O, O, B_<quý>, B_<năm>, O, O, O, O, B_<tháng>, B_<năm>, O, O, O, O, O, O, O, O, B_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, I_<tên_cụm_chỉ_tiêu>, O, O, O, O, O, O, O, O, O, B_<tên_tổng_công_ty>]"
5980,Cho t hỏi trong quý 4 năm 2021 tính đến th 12 năm 2021 tầm nhìn tổng_hợp như_thế_nào về nhóm chỉ_tiêu Mức_độ hài_lòng (theo từng dịch_vụ trọng_điểm) của tổng_công_ty VTS,"[O, O, O, O, O, B_<quý>, O, B_<năm>,

#CRFsuite

In [ ]:
!pip install sklearn-crfsuite

In [ ]:
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import classification_report, f1_score

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

In [ ]:
nltk.download('conll2002')

In [ ]:
nltk.corpus.conll2002.fileids()

In [ ]:
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

In [ ]:
len(train_sents), len(test_sents), sum([len(each) for each in train_sents]), sum([len(each) for each in test_sents])

In [ ]:
train_sents

In [ ]:
postag = "N"

In [ ]:
word = train_sents[0][1][0]
postag = train_sents[0][1][1]
sent = train_sents[0]
i = 1

In [ ]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [ ]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [ ]:
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
try:
    crf.fit(X_train, y_train)
except AttributeError:
    pass
# predictions = crf.predict(X_test)

In [ ]:
labels = list(crf.classes_)
labels.remove('O')
# labels
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='', labels=labels)

In [ ]:
def handle_arr(arr):
    ret = []
    for each in arr:
        ret = [*ret, *each]
    return ret

y_pred = handle_arr(y_pred)
y_test = handle_arr(y_test)

In [ ]:
print(classification_report(
    y_test, y_pred, labels=labels
))